# Try to use OCR to recognize the letters in the Wordle game

In [ ]:
# Load test image
from IPython.display import Image, display

# Show the image
display(Image(filename='images/small_letters.png'))

In [1]:
from image_processing import get_wordle_grid_boxes, crop_cell_margin, detect_letter, tesseract_inference
import cv2

image = cv2.imread('images/row 2.PNG')

rows = get_wordle_grid_boxes(image)

if rows:
    print(f"Extracted {len(rows)} rows from the Wordle board")

    # Crop the cells and put them into a nested list
    wordle_board_imgs = []
    for row in rows:
        wordle_board_row = []
        for cell in row:
            x, y, w, h = cell[1:5]
            cropped_cell = image[y:y+h, x:x+w]
            cropped_cell = crop_cell_margin(cropped_cell)
            wordle_board_row.append(cropped_cell)
        wordle_board_imgs.append(wordle_board_row)


    # Run Tesseract OCR on each cell
    for i, row in enumerate(wordle_board_imgs):
        for j, cell in enumerate(row):
            if detect_letter(cell.copy()):
                # Show the thresholded image
                cv2.imshow("Croped Image", cell)
                cv2.waitKey(0)
                cv2.destroyAllWindows()


                letter = tesseract_inference(cell)
                print(f"Row {i+1}, Column {j+1}: {letter}")
            else:
                print(f"Row {i+1}, Column {j+1}: No letter detected")
else:
    print(f"Failed to extract wordle board, found {len(rows)} rows instead of 6")

Extracted 6 rows from the Wordle board
Row 1, Column 1: T
Row 1, Column 2: A
Row 1, Column 3: L
Row 1, Column 4: E
Row 1, Column 5: S
Row 2, Column 1: None
Row 2, Column 2: A
Row 2, Column 3: T
Row 2, Column 4: E
Row 2, Column 5: None
Row 3, Column 1: No letter detected
Row 3, Column 2: No letter detected
Row 3, Column 3: No letter detected
Row 3, Column 4: No letter detected
Row 3, Column 5: No letter detected
Row 4, Column 1: No letter detected
Row 4, Column 2: No letter detected
Row 4, Column 3: No letter detected
Row 4, Column 4: No letter detected
Row 4, Column 5: No letter detected
Row 5, Column 1: No letter detected
Row 5, Column 2: No letter detected
Row 5, Column 3: No letter detected
Row 5, Column 4: No letter detected
Row 5, Column 5: No letter detected
Row 6, Column 1: No letter detected
Row 6, Column 2: No letter detected
Row 6, Column 3: No letter detected
Row 6, Column 4: No letter detected
Row 6, Column 5: No letter detected


# Also extract the colors from the Wordle board